# Segment Anything

In [1]:
import matplotlib.pyplot as plt
from skimage import io
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

In [2]:
import os.path as op
IMG_DIR = op.expanduser("/Users/bao/Dropbox/Manuscripts/GRABSEEDS/leaf_example")

In [ ]:
test_img = io.imread(op.join(IMG_DIR, "leaf.jpg"))